In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json

In [15]:
url = "https://www.imdb.com/chart/moviemeter/" #sending GET request for most popular movies
headers = {"User-Agent":"Chrome/131.0.6778.109"}
response = requests.get(url, headers=headers)
response

<Response [200]>

In [17]:
if response.status_code == 200:
    print("Successfully fetched the page!")
else:
    print("Failed to retrieve the page. Status code: {response.status_code}")

Successfully fetched the page!


In [19]:
soup = BeautifulSoup(response.text, 'html.parser')
soup

<!DOCTYPE html>
<html lang="en-US" xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://opengraphprotocol.org/schema/"><head><meta charset="utf-8"/><meta content="width=device-width" name="viewport"/><script>if(typeof uet === 'function'){ uet('bb', 'LoadTitle', {wb: 1}); }</script><script>window.addEventListener('load', (event) => {
        if (typeof window.csa !== 'undefined' && typeof window.csa === 'function') {
            var csaLatencyPlugin = window.csa('Content', {
                element: {
                    slotId: 'LoadTitle',
                    type: 'service-call'
                }
            });
            csaLatencyPlugin('mark', 'clickToBodyBegin', 1733706957999);
        }
    })</script><title>Most Popular Movies</title><meta content="As determined by IMDb users" data-id="main" name="description"/><script type="application/ld+json">{"@type":"ItemList","itemListElement":[{"@type":"ListItem","item":{"@type":"Movie","url":"https://www.imdb.com/title/tt1262

In [21]:
print(soup.prettify()[:2000])

<!DOCTYPE html>
<html lang="en-US" xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://opengraphprotocol.org/schema/">
 <head>
  <meta charset="utf-8"/>
  <meta content="width=device-width" name="viewport"/>
  <script>
   if(typeof uet === 'function'){ uet('bb', 'LoadTitle', {wb: 1}); }
  </script>
  <script>
   window.addEventListener('load', (event) => {
        if (typeof window.csa !== 'undefined' && typeof window.csa === 'function') {
            var csaLatencyPlugin = window.csa('Content', {
                element: {
                    slotId: 'LoadTitle',
                    type: 'service-call'
                }
            });
            csaLatencyPlugin('mark', 'clickToBodyBegin', 1733706957999);
        }
    })
  </script>
  <title>
   Most Popular Movies
  </title>
  <meta content="As determined by IMDb users" data-id="main" name="description"/>
  <script type="application/ld+json">
   {"@type":"ItemList","itemListElement":[{"@type":"ListItem","item":{"@type":

In [27]:
script_tag = soup.find('script', type='application/ld+json')

In [33]:
if script_tag:
    data = json.loads(script_tag.string)

In [35]:
titles = []
urls = []
descriptions = []
best_ratings = []
worst_ratings = []
rating_values = []
genres = []
durations = []

In [43]:
if 'itemListElement' in data:
    for item in data['itemListElement']:
        movie = item['item']
        
        titles.append(movie.get('name', 'N/A'))  # Movie title
        urls.append(movie.get('url', 'N/A'))     # IMDb page URL
        descriptions.append(movie.get('description', 'N/A'))  # Movie description
        best_ratings.append(movie.get('aggregateRating', {}).get('bestRating', 'N/A'))  # Best possible rating
        worst_ratings.append(movie.get('aggregateRating', {}).get('worstRating', 'N/A'))  # Worst possible rating
        rating_values.append(movie.get('aggregateRating', {}).get('ratingValue', 'N/A'))  # Current movie rating
        genres.append(movie.get('genre', 'N/A'))  # Movie genre(s)
        durations.append(movie.get('duration', 'N/A'))  # Movie duration



In [45]:
df = pd.DataFrame({
    'Title': titles,              # Column for movie titles
    'URL': urls,                  # Column for IMDb URLs
    'Description': descriptions,  # Column for movie descriptions
    'Best Rating': best_ratings,  # Column for the best possible rating (usually 10)
    'Worst Rating': worst_ratings, # Column for the worst possible rating (usually 1)
    'Rating Value': rating_values, # Column for the actual user rating
    'Genre': genres,              # Column for the movie's genre (e.g., Drama, Comedy)
    'Duration': durations         # Column for the movie's duration (in ISO format, e.g., PT2H30M)
})

In [47]:
df.to_csv('most_popular_movies.csv', index=False)  # index=False means we do not save the index column
print("Data saved to 'most_popular_movies.csv'")

Data saved to 'most_popular_movies.csv'
